In [1]:
import os
import subprocess
import platform
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageOps
import cv2
import numpy as np
from scipy.spatial import distance
import math
from datetime import datetime
import tempfile
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import gc
import imageio
import time
import re
import io
import sys

class PreciseFrameExtractor:
    """
    A class specifically designed to extract frames from contact sheets with registration marks
    at the corners of each frame. This implementation focuses on precise content extraction
    and proper centering.
    """
    
    def __init__(self):
        self.debug_mode = False
        self.min_frame_size = 100
        self.max_frames = 50  # Changed from 100 to match expected frame count
        self.registration_mark_size = 20
        self.margin_percent = 0.05
    
    def set_debug_mode(self, debug):
        """Set debug mode."""
        self.debug_mode = debug
    
    def extract_frames(self, image, frames_per_row=None):
        """
        Extract frames from a contact sheet image using multiple adaptive approaches.
        
        Args:
            image: Input image (BGR format from OpenCV)
            frames_per_row: Number of frames per row (auto-detected if None)
            
        Returns:
            List of extracted frames as OpenCV images (BGR format)
        """
        # Check if image is valid
        if image is None or image.size == 0 or len(image.shape) < 2:
            print("Warning: Invalid input image for frame extraction")
            return []
            
        # Create a copy for visualization if debug mode is on
        debug_img = image.copy() if self.debug_mode else None
        
        # Try different approaches to extract frames
        frames = []
        
        # Approach 1: Registration mark detection
        print("Attempting registration mark detection...")
        try:
            frames = self._extract_frames_from_registration_marks(image)
            if frames and len(frames) >= 2:  # Need at least a couple of frames
                print(f"Successfully extracted {len(frames)} frames using registration mark detection")
                return frames
        except Exception as e:
            print(f"Registration mark detection failed: {str(e)}")
        
        # Approach 2: Grid-based extraction
        print("Trying grid-based extraction...")
        try:
            frames = self._extract_frames_grid_based(image, frames_per_row)
            if frames and len(frames) >= 2:
                print(f"Successfully extracted {len(frames)} frames using grid-based approach")
                return frames
        except Exception as e:
            print(f"Grid-based extraction failed: {str(e)}")
        
        # Approach 3: Content-based extraction
        print("Attempting content-based extraction...")
        try:
            # Convert to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Try multiple thresholding methods
            thresholds = [
                # Binary threshold
                lambda g: cv2.threshold(g, 220, 255, cv2.THRESH_BINARY_INV)[1],
                # Adaptive threshold
                lambda g: cv2.adaptiveThreshold(g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY_INV, 21, 10),
                # Otsu's threshold
                lambda g: cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            ]
            
            content_frames = []
            for threshold_func in thresholds:
                if content_frames:
                    break  # Stop if we already found frames
                    
                try:
                    binary = threshold_func(gray)
                    
                    # Find contours
                    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    
                    # Determine reasonable size constraints based on image size
                    height, width = image.shape[:2]
                    min_area = (width * height) / 200  # Frames should be at least 0.5% of image
                    max_area = (width * height) / 6    # Frames shouldn't be larger than 1/6 of image
                    
                    # Filter contours by size
                    valid_contours = [c for c in contours if min_area < cv2.contourArea(c) < max_area]
                    
                    # Extract frames from valid contours
                    for c in valid_contours:
                        x, y, w, h = cv2.boundingRect(c)
                        
                        # Skip if aspect ratio is too extreme
                        aspect_ratio = w / float(h)
                        if aspect_ratio < 0.5 or aspect_ratio > 2:
                            continue
                        
                        # Extract frame
                        frame = image[y:y+h, x:x+w]
                        
                        # Center the frame
                        frame = self._center_frame_cv2(frame)
                        content_frames.append(frame)
                        
                        # Draw in debug mode
                        if self.debug_mode and debug_img is not None:
                            cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0, 0, 255), 2)
                except Exception as e:
                    print(f"Threshold method failed: {str(e)}")
                    continue
            
            if content_frames and len(content_frames) >= 2:
                print(f"Successfully extracted {len(content_frames)} frames using content-based approach")
                
                # Save debug image
                if self.debug_mode and debug_img is not None:
                    cv2.imwrite("debug_content_extraction.jpg", debug_img)
                    
                return content_frames
        except Exception as e:
            print(f"Content-based extraction failed: {str(e)}")
        
        # Approach 4: Simple division (last resort)
        print("Falling back to simple division approach...")
        try:
            # Use frames_per_row if provided, otherwise make a reasonable guess
            cols = frames_per_row if frames_per_row else 4  # Default to 4 columns
            
            # Estimate rows based on image aspect ratio
            height, width = image.shape[:2]
            aspect_ratio = height / width
            rows = max(1, int(cols * aspect_ratio * 0.75))  # Adjust for typical frame aspect ratios
            
            # Calculate margins and cell dimensions
            margin_x = int(width * 0.05)  # 5% margin
            margin_y = int(height * 0.05)  # 5% margin
            cell_width = (width - 2 * margin_x) // cols
            cell_height = (height - 2 * margin_y) // rows
            
            # Inner margin to exclude registration marks
            inner_margin_percent = 0.15  # Default 15% inner margin
            
            simple_frames = []
            for row in range(rows):
                for col in range(cols):
                    # Calculate frame coordinates
                    x1 = margin_x + col * cell_width
                    y1 = margin_y + row * cell_height
                    x2 = x1 + cell_width
                    y2 = y1 + cell_height
                    
                    # Apply inner margin
                    inner_margin_x = int(cell_width * inner_margin_percent)
                    inner_margin_y = int(cell_height * inner_margin_percent)
                    
                    x1 += inner_margin_x
                    y1 += inner_margin_y
                    x2 -= inner_margin_x
                    y2 -= inner_margin_y
                    
                    # Ensure coordinates are within image bounds
                    x1 = max(0, x1)
                    y1 = max(0, y1)
                    x2 = min(width, x2)
                    y2 = min(height, y2)
                    
                    # Skip if resulting frame is too small
                    if x2 - x1 < self.min_frame_size or y2 - y1 < self.min_frame_size:
                        continue
                    
                    # Extract frame
                    frame = image[y1:y2, x1:x2]
                    
                    # Center the frame
                    frame = self._center_frame_cv2(frame)
                    simple_frames.append(frame)
                    
                    # Limit number of frames
                    if len(simple_frames) >= self.max_frames:
                        print(f"Reached maximum frame limit of {self.max_frames}")
                        return simple_frames
            
            if simple_frames:
                print(f"Successfully extracted {len(simple_frames)} frames using simple division")
                return simple_frames
        except Exception as e:
            print(f"Simple division failed: {str(e)}")
        
        # If all approaches failed, return empty list
        print("All extraction approaches failed")
        return []
    
    def _extract_frames_grid_based(self, img, frames_per_row=None):
        """
        Extract frames using a precise grid-based approach with content-aware positioning.
        """
        height, width = img.shape[:2]
        
        # Detect grid parameters
        rows, cols, cell_width, cell_height, margin_x, margin_y = self._detect_grid_parameters(img, frames_per_row)
        
        if self.debug_mode:
            print(f"Detected grid: {rows}x{cols}, cell size: {cell_width}x{cell_height}, margins: {margin_x},{margin_y}")
            debug_img = img.copy()
        
        # Extract frames from grid
        frames = []
        for row in range(rows):
            for col in range(cols):
                # Calculate cell boundaries
                x1 = margin_x + col * cell_width
                y1 = margin_y + row * cell_height
                x2 = x1 + cell_width
                y2 = y1 + cell_height
                
                # Ensure coordinates are within image bounds
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(width, x2)
                y2 = min(height, y2)
                
                # Extract the cell
                cell = img[y1:y2, x1:x2]
                
                # Find the actual content within the cell
                gray_cell = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)
                _, binary = cv2.threshold(gray_cell, 240, 255, cv2.THRESH_BINARY_INV)
                
                # Find contours of the content
                contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                # Find bounding box of all content
                if contours:
                    # Combine all contours to find the overall content area
                    all_contours = np.vstack([contour for contour in contours])
                    x, y, w, h = cv2.boundingRect(all_contours)
                    
                    # Add small margin around content
                    margin = int(min(w, h) * 0.1)
                    content_x1 = max(0, x - margin)
                    content_y1 = max(0, y - margin)
                    content_x2 = min(cell.shape[1], x + w + margin)
                    content_y2 = min(cell.shape[0], y + h + margin)
                    
                    # Extract content if it's a reasonable size
                    if w > cell.shape[1] * 0.1 and h > cell.shape[0] * 0.1:
                        content = cell[content_y1:content_y2, content_x1:content_x2]
                        
                        # Center the content in a square canvas
                        frame = self._center_frame_cv2(content)
                        frames.append(frame)
                        continue
                
                # If content detection failed, fall back to standard inner margin approach
                inner_margin_percent = 0.22  # Increased to better exclude registration marks
                inner_margin = int(min(cell_width, cell_height) * inner_margin_percent)
                
                # Apply inner margin
                frame_x1 = x1 + inner_margin
                frame_y1 = y1 + inner_margin
                frame_x2 = x2 - inner_margin
                frame_y2 = y2 - inner_margin
                
                # Ensure frame coordinates are valid
                frame_x1 = max(0, frame_x1)
                frame_y1 = max(0, frame_y1)
                frame_x2 = min(width, frame_x2)
                frame_y2 = min(height, frame_y2)
                
                # Skip if resulting frame is too small
                if frame_x2 - frame_x1 < self.min_frame_size or frame_y2 - frame_y1 < self.min_frame_size:
                    continue
                
                # Extract and center frame
                frame = img[frame_y1:frame_y2, frame_x1:frame_x2]
                frame = self._center_frame_cv2(frame)
                frames.append(frame)
        
        return frames


    
    def _detect_circular_marks(self, img):
        """Specifically detect circular registration marks with improved parameters."""
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Reduced kernel size for finer details
        
        # Adjusted parameters based on your specific registration marks
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.5,  # Reduced dp for more precision
                                minDist=50,  # Decreased minimum distance
                                param1=100,   # Reduced edge detection threshold
                                param2=50,    # Reduced accumulator threshold to detect more circles
                                minRadius=10, # Reduced min radius to catch smaller marks
                                maxRadius=30) # Adjusted max radius
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            # Limit to a reasonable number of circles
            max_circles = 100  # Increased maximum circles
            if len(circles[0]) > max_circles:
                # Take only the strongest matches if too many are found
                circles = np.array([circles[0][:max_circles]])
            return [(x, y) for x, y, r in circles[0, :]]
        return []

    def _detect_grid_parameters(self, img, frames_per_row=None):
        """
        Improved detection of grid parameters for better frame extraction.
        """
        height, width = img.shape[:2]
        
        # Convert to grayscale and threshold
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Try multiple thresholding approaches for better detection
        binary = None
        thresholds = [
            lambda g: cv2.threshold(g, 200, 255, cv2.THRESH_BINARY_INV)[1],
            lambda g: cv2.adaptiveThreshold(g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2),
            lambda g: cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        ]
        
        valid_contours = []
        for threshold_func in thresholds:
            try:
                binary = threshold_func(gray)
                
                # Find contours
                contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                # Filter contours by size
                min_contour_area = (width * height) / 2000  # Reduced for smaller frames
                temp_valid_contours = [c for c in contours if cv2.contourArea(c) > min_contour_area]
                
                if len(temp_valid_contours) >= 4:
                    valid_contours = temp_valid_contours
                    break
            except Exception as e:
                print(f"Threshold method failed: {str(e)}")
                continue
        
        # Calculate grid parameters based on contour distribution
        if len(valid_contours) >= 4:
            # Get bounding boxes and their centers
            bboxes = [cv2.boundingRect(c) for c in valid_contours]
            centers = [(x + w//2, y + h//2) for (x, y, w, h) in bboxes]
            
            # Cluster centers to find rows and columns
            y_coords = np.array([c[1] for c in centers]).reshape(-1, 1)
            # Use a smaller eps for more precise row detection
            row_clustering = DBSCAN(eps=height/30, min_samples=1).fit(y_coords)
            
            # Calculate row count from unique labels
            unique_rows = len(np.unique(row_clustering.labels_))
            rows = unique_rows
            
            # Calculate column count using x-coordinate clustering
            if frames_per_row is None:
                x_coords = np.array([c[0] for c in centers]).reshape(-1, 1)
                # Use a smaller eps for more precise column detection
                col_clustering = DBSCAN(eps=width/30, min_samples=1).fit(x_coords)
                cols = len(np.unique(col_clustering.labels_))
            else:
                cols = frames_per_row
            
            # Calculate cell size based on median dimensions with adjustment
            widths = [w for (x, y, w, h) in bboxes]
            heights = [h for (x, y, w, h) in bboxes]
            
            # Use percentile instead of median for more robust estimation
            # 75th percentile gives a better estimate for cell size
            cell_width = int(np.percentile(widths, 75) * 1.6)  # Increased multiplier
            cell_height = int(np.percentile(heights, 75) * 1.6)
            
            # Calculate dynamic margins
            margin_x = max(20, int((width - (cols * cell_width)) / 2))
            margin_y = max(20, int((height - (rows * cell_height)) / 2))
            
            return rows, cols, cell_width, cell_height, margin_x, margin_y
        
        # Fallback to aspect ratio estimation - improved with better defaults
        aspect_ratio = width / height
        
        # Let's handle a special case for your specific contact sheet shown in the image
        # It appears to be a 4x4 grid (or similar)
        if frames_per_row is None:
            # Default to 4 columns if your contact sheet typically has 4 columns
            cols = 4
        else:
            cols = frames_per_row
            
        # Calculate rows based on aspect ratio
        # For the sample shown, it looks like there are multiple rows
        rows = int((cols * height) / (width * aspect_ratio))
        # Ensure we have at least 2 rows
        rows = max(2, rows)
        
        # Calculate cell size with improved margins
        margin_x = int(width * 0.05)
        margin_y = int(height * 0.05)
        cell_width = (width - 2 * margin_x) // cols
        cell_height = (height - 2 * margin_y) // rows
        
        return rows, cols, cell_width, cell_height, margin_x, margin_y
    
            
    def _extract_frames_from_registration_marks(self, img):
        """Extract frames by detecting registration marks with adaptive parameter selection."""
        height, width = img.shape[:2]
        
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Create a debug image if debug mode is on
        debug_img = img.copy() if self.debug_mode else None
        
        # Try multiple thresholds to improve registration mark detection
        all_reg_mark_centers = []
        
        # Use adaptive thresholds based on image characteristics
        mean_val = np.mean(gray)
        std_val = np.std(gray)
        
        # Dynamic thresholds based on image statistics
        thresholds = [
            # Brighter threshold for light images
            lambda g: cv2.threshold(g, min(mean_val + std_val, 200), 255, cv2.THRESH_BINARY_INV)[1],
            # Middle threshold
            lambda g: cv2.threshold(g, mean_val, 255, cv2.THRESH_BINARY_INV)[1],
            # Darker threshold for dark images
            lambda g: cv2.threshold(g, max(mean_val - std_val, 50), 255, cv2.THRESH_BINARY_INV)[1],
            # Adaptive threshold
            lambda g: cv2.adaptiveThreshold(g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY_INV, 11, 2),
            # Otsu's threshold
            lambda g: cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        ]
        
        for threshold_func in thresholds:
            try:
                binary = threshold_func(gray)
                
                # Find contours
                contours, _ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                
                # Calculate dynamic size limits based on image dimensions
                min_area = (width * height) / 10000  # Adaptive minimum size
                max_area = (width * height) / 200    # Adaptive maximum size
                
                # Filter contours by size and shape
                for c in contours:
                    area = cv2.contourArea(c)
                    
                    if area < min_area or area > max_area:
                        continue
                    
                    # Check if contour is circular or square (registration marks can be either)
                    perimeter = cv2.arcLength(c, True)
                    if perimeter == 0:
                        continue
                    
                    circularity = 4 * np.pi * area / (perimeter * perimeter)
                    # Accept shapes that are somewhat circular or square (0.5-0.9 covers most marks)
                    if 0.5 <= circularity <= 0.9:
                        # Get center of contour
                        M = cv2.moments(c)
                        if M["m00"] != 0:
                            cx = int(M["m10"] / M["m00"])
                            cy = int(M["m01"] / M["m00"])
                            all_reg_mark_centers.append((cx, cy))
                            
                            # Draw in debug mode
                            if self.debug_mode and debug_img is not None:
                                cv2.drawContours(debug_img, [c], -1, (0, 0, 255), 2)
                                cv2.circle(debug_img, (cx, cy), 3, (255, 0, 0), -1)
            except Exception as e:
                print(f"Threshold method failed: {str(e)}")
                continue
        
        # Remove duplicate centers
        unique_centers = []
        for center in all_reg_mark_centers:
            is_duplicate = False
            for unique in unique_centers:
                # Dynamic distance threshold based on image size
                distance_threshold = min(width, height) / 100
                if math.sqrt((center[0] - unique[0])**2 + (center[1] - unique[1])**2) < distance_threshold:
                    is_duplicate = True
                    break
            if not is_duplicate:
                unique_centers.append(center)
        
        if len(unique_centers) < 4:
            raise ValueError(f"Not enough registration marks detected (found {len(unique_centers)})")
        
        # Group centers into frames using DBSCAN with adaptive epsilon
        frames = []
        centers_array = np.array(unique_centers)
        
        # Estimate epsilon based on image size and mark distribution
        # Find min distance between points to estimate mark spacing
        all_dists = []
        for i in range(len(unique_centers)):
            for j in range(i+1, len(unique_centers)):
                dist = math.sqrt((unique_centers[i][0] - unique_centers[j][0])**2 + 
                            (unique_centers[i][1] - unique_centers[j][1])**2)
                all_dists.append(dist)
        
        # Sort distances and get the smallest ones (likely within same frame)
        if all_dists:
            all_dists.sort()
            # Use a percentile approach to estimate frame size
            frame_size_est = np.percentile(all_dists, 75)
            epsilon = frame_size_est * 1.2  # Epsilon slightly larger than estimated frame size
        else:
            # Fallback if no distances calculated
            epsilon = min(width, height) / 5
        
        # Adjust min_samples based on expected frame structure
        # 4 corners = 4 marks, but we'll accept 3 to handle partial marks
        min_samples = 3
        
        # Perform clustering
        clustering = DBSCAN(eps=epsilon, min_samples=min_samples).fit(centers_array)
        
        # Get unique labels
        labels = clustering.labels_
        unique_labels = set(labels)
        
        # Process clusters to find frames
        for label in unique_labels:
            if label == -1:  # Skip noise points
                continue
                
            # Get points in this cluster
            cluster_points = centers_array[labels == label]
            points = [(int(p[0]), int(p[1])) for p in cluster_points]
            
            # We need at least 3 points to estimate a frame
            if len(points) < 3:
                continue
            
            # For 3 or more points, estimate a rectangular frame
            # We'll use convex hull and minimum area rectangle
            hull = cv2.convexHull(np.array(points))
            rect = cv2.minAreaRect(hull)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            frame_corners = [tuple(p) for p in box]
            
            # Sort the corners (top-left, top-right, bottom-right, bottom-left)
            sorted_corners = self._sort_points(frame_corners)
            
            # Calculate inner margin based on frame size
            width_points = int((sorted_corners[1][0] - sorted_corners[0][0] + 
                            sorted_corners[2][0] - sorted_corners[3][0]) / 2)
            height_points = int((sorted_corners[3][1] - sorted_corners[0][1] + 
                            sorted_corners[2][1] - sorted_corners[1][1]) / 2)
            
            # Use adaptive inner margin based on frame size
            inner_margin_percent = 0.15  # Default value
            
            # Calculate frame boundaries
            center_x = int(sum(p[0] for p in sorted_corners) / 4)
            center_y = int(sum(p[1] for p in sorted_corners) / 4)
            
            frame_width = int(width_points * (1 - 2 * inner_margin_percent))
            frame_height = int(height_points * (1 - 2 * inner_margin_percent))
            
            x1 = center_x - frame_width // 2
            y1 = center_y - frame_height // 2
            x2 = x1 + frame_width
            y2 = y1 + frame_height
            
            # Draw in debug mode
            if self.debug_mode and debug_img is not None:
                for p in sorted_corners:
                    cv2.circle(debug_img, p, 5, (0, 255, 0), -1)
                cv2.rectangle(debug_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            # Ensure coordinates are within image bounds
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(width, x2)
            y2 = min(height, y2)
            
            # Skip if resulting frame is too small
            min_size = min(width, height) / 20  # Adaptive minimum size
            if x2 - x1 < min_size or y2 - y1 < min_size:
                continue
            
            # Extract and center frame
            frame = img[y1:y2, x1:x2]
            frame = self._center_frame_cv2(frame)
            frames.append(frame)
        
        # Save debug image
        if self.debug_mode and debug_img is not None:
            cv2.imwrite("debug_registration_marks.jpg", debug_img)
        
        return frames




    def _sort_points(self, points):
        """Sort points in order: top-left, top-right, bottom-right, bottom-left."""
        # Sort points based on x-coordinate first
        points = sorted(points, key=lambda p: p[0])
        
        # Get the left points (first 2) and right points (last 2)
        left_points = points[:2]
        right_points = points[2:] if len(points) >= 4 else points[1:]
        
        # Sort left points by y-coordinate
        left_points = sorted(left_points, key=lambda p: p[1])
        top_left, bottom_left = left_points if len(left_points) >= 2 else (left_points[0], left_points[0])
        
        # Sort right points by y-coordinate
        right_points = sorted(right_points, key=lambda p: p[1])
        top_right, bottom_right = right_points if len(right_points) >= 2 else (right_points[0], right_points[0])
        
        return [top_left, top_right, bottom_right, bottom_left]


    def _is_rectangle(self, points):
        """Check if points form a rectangle with adaptive tolerance."""
        if len(points) != 4:
            return False
        
        # Calculate pairwise distances between points
        distances = []
        for i in range(len(points)):
            for j in range(i+1, len(points)):
                dx = points[i][0] - points[j][0]
                dy = points[i][1] - points[j][1]
                distances.append(math.sqrt(dx*dx + dy*dy))
        
        # Sort distances
        distances.sort()
        
        # For a rectangle, we should have 4 equal sides and 2 equal diagonals
        if len(distances) != 6:  # Should have exactly 6 distances for 4 points
            return False
        
        # Group similar distances (sides and diagonals)
        # Use a hierarchical approach to group similar distances
        from scipy.cluster.hierarchy import linkage, fcluster
        
        # Convert to numpy array for clustering
        distances_array = np.array(distances).reshape(-1, 1)
        
        # Perform hierarchical clustering on distances
        if len(distances_array) >= 2:  # Need at least 2 distances for clustering
            Z = linkage(distances_array, 'single')
            # Try to cluster into 2 groups (sides and diagonals)
            try:
                clusters = fcluster(Z, 2, criterion='maxclust')
                # Count elements in each cluster
                counts = np.bincount(clusters)
                # For a rectangle: one cluster should have 4 elements (sides), other should have 2 (diagonals)
                if len(counts) > 1 and (counts[0] == 4 and counts[1] == 2 or counts[0] == 2 and counts[1] == 4):
                    return True
            except:
                # Fall back to simpler approach if clustering fails
                pass
        
        # Fallback approach: check variance within sides and diagonals
        sides = distances[:4]
        diagonals = distances[4:]
        
        # Calculate variance within sides and diagonals
        sides_mean = sum(sides) / len(sides)
        sides_variance = sum((x - sides_mean) ** 2 for x in sides) / len(sides)
        sides_relative_variance = sides_variance / (sides_mean ** 2)
        
        diagonals_mean = sum(diagonals) / len(diagonals)
        diagonals_variance = sum((x - diagonals_mean) ** 2 for x in diagonals) / len(diagonals)
        diagonals_relative_variance = diagonals_variance / (diagonals_mean ** 2)
        
        # Adaptive tolerance based on image characteristics
        # For a perfect rectangle, both variances would be 0
        # Allow more variance for real-world images with perspective distortion
        if sides_relative_variance < 0.1 and diagonals_relative_variance < 0.1:
            return True
        
        return False


    
    def _extract_frames_simple_division(self, img, frames_per_row):
        """
        Extract frames by simply dividing the image into a grid.
        """
        height, width = img.shape[:2]
        
        # Estimate total frames (assume 9 rows x 4 columns = 36 frames by default)
        total_frames = 36
        rows = total_frames // frames_per_row
        
        # Calculate cell size and margins
        margin_percent = 0.05  # 5% margin
        margin_x = int(width * margin_percent)
        margin_y = int(height * margin_percent)
        cell_width = (width - 2 * margin_x) // frames_per_row
        cell_height = (height - 2 * margin_y) // rows
        
        # Extract frames from grid
        frames = []
        for row in range(rows):
            for col in range(frames_per_row):
                # Calculate frame coordinates with margins
                x1 = margin_x + col * cell_width
                y1 = margin_y + row * cell_height
                x2 = x1 + cell_width
                y2 = y1 + cell_height
                
                # Add inner margin to exclude registration marks
                inner_margin = int(min(cell_width, cell_height) * 0.1)
                x1 += inner_margin
                y1 += inner_margin
                x2 -= inner_margin
                y2 -= inner_margin
                
                # Ensure coordinates are within image bounds
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(width, x2)
                y2 = min(height, y2)
                
                # Skip if resulting frame is too small
                if x2 - x1 < self.min_frame_size or y2 - y1 < self.min_frame_size:
                    continue
                
                # Extract frame
                frame = img[y1:y2, x1:x2]
                
                # Center the frame in a square canvas
                frame = self._center_frame_cv2(frame)
                
                frames.append(frame)
                
                # Limit number of frames
                if len(frames) >= self.max_frames:
                    return frames
        
        return frames
            
    def _center_frame_cv2(self, frame):
        """Center the frame content in a square canvas with white background."""
        if frame is None or frame.size == 0:
            return None
            
        # Get frame dimensions
        height, width = frame.shape[:2]
        
        # Find the actual content boundaries
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours to identify content areas
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Determine where to place the frame based on content
        content_x1, content_y1, content_width, content_height = 0, 0, width, height
        
        if contours:
            # Find bounding rectangle of all content
            all_contours = np.vstack([contour for contour in contours])
            content_x1, content_y1, content_width, content_height = cv2.boundingRect(all_contours)
        
        # Determine the size of the square canvas (max of content width and height plus padding)
        max_content_dim = max(content_width, content_height)
        padding = int(max_content_dim * 0.2)  # 20% padding
        canvas_size = max_content_dim + 2 * padding
        
        # Create a new square image with white background
        centered_frame = np.ones((canvas_size, canvas_size, 3), dtype=np.uint8) * 255
        
        # Calculate position to paste the content (centered)
        paste_x = (canvas_size - content_width) // 2
        paste_y = (canvas_size - content_height) // 2
        
        # Paste the content onto the canvas
        if content_width > 0 and content_height > 0:
            content = frame[content_y1:content_y1+content_height, content_x1:content_x1+content_width]
            centered_frame[paste_y:paste_y+content_height, paste_x:paste_x+content_width] = content
        
        return centered_frame



    def _visualize_extraction(self, img, centers, frames, filename="debug_extraction.jpg"):
        """
        Create a visualization of the extraction process for debugging.
        
        Args:
            img: Original image
            centers: List of detected registration mark centers
            frames: List of extracted frame boundaries [(x1,y1,x2,y2), ...]
            filename: Output filename for the debug image
        """
        if not self.debug_mode:
            return
            
        debug_img = img.copy()
        
        # Draw all detected registration mark centers
        for center in centers:
            cv2.circle(debug_img, center, 5, (0, 0, 255), -1)
        
        # Draw frame boundaries
        for i, (x1, y1, x2, y2) in enumerate(frames):
            cv2.rectangle(debug_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(debug_img, f"{i+1}", (x1+10, y1+30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Save the debug image
        cv2.imwrite(filename, debug_img)

class AnimationApp:
    """
    Application for extracting frames from contact sheets and creating animations.
    """
    
    def __init__(self, root):
        self.root = root
        self.root.title("Animation Extractor")
        
        # Set up variables
        self.contact_sheet = None
        self.extracted_frames = []
        self.current_frame_index = 0
        self.is_playing = False
        self.play_after_id = None
        
        # Create UI variables
        self.status_var = tk.StringVar(value="Ready")
        self.frame_rate = tk.DoubleVar(value=12.0)  # Default 12 FPS
        self.frames_per_row = tk.IntVar(value=4)  # Default 4 frames per row
        self.extraction_mode = tk.StringVar(value="auto")  # Default to auto mode
        self.registration_mark_threshold = tk.IntVar(value=70)  # Default 70%
        self.debug_mode = tk.BooleanVar(value=False)  # Debug mode toggle
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create left panel for controls
        control_frame = ttk.Frame(main_frame, padding="5", width=200)
        control_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # Create right panel for image display
        display_frame = ttk.Frame(main_frame)
        display_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Create canvas for image display
        self.canvas = tk.Canvas(display_frame, bg="#f0f0f0", highlightthickness=0)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Input section
        input_frame = ttk.LabelFrame(control_frame, text="Input", padding="5")
        input_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(input_frame, text="Load Contact Sheet", command=self.load_contact_sheet).pack(fill=tk.X)
        ttk.Button(input_frame, text="Extract Frames", command=self.extract_frames_from_registration_marks).pack(fill=tk.X, pady=(5, 0))
        
        # Extraction mode section
        mode_frame = ttk.LabelFrame(control_frame, text="Extraction Settings", padding="5")
        mode_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Mode selection
        ttk.Label(mode_frame, text="Mode:").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="Auto", variable=self.extraction_mode, value="auto").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="Digital", variable=self.extraction_mode, value="digital").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="RISO Print", variable=self.extraction_mode, value="riso").pack(anchor=tk.W)
        
        # Frames per row setting
        ttk.Label(mode_frame, text="Frames Per Row:").pack(anchor=tk.W, pady=(5, 0))
        ttk.Spinbox(mode_frame, from_=1, to=20, textvariable=self.frames_per_row, width=5).pack(anchor=tk.W)
        
        # Registration mark threshold
        ttk.Label(mode_frame, text="Registration Mark Threshold (%):").pack(anchor=tk.W, pady=(5, 0))
        ttk.Spinbox(mode_frame, from_=10, to=90, textvariable=self.registration_mark_threshold, width=5).pack(anchor=tk.W)
        
        # Debug mode toggle
        ttk.Checkbutton(mode_frame, text="Debug Mode", variable=self.debug_mode).pack(anchor=tk.W, pady=(5, 0))
        
        # Extract button
        ttk.Button(mode_frame, text="Extract Frames", command=self.extract_frames_from_registration_marks).pack(fill=tk.X, pady=(5, 0))
        
        # Playback controls
        playback_frame = ttk.LabelFrame(control_frame, text="Playback", padding="5")
        playback_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Frame rate control
        ttk.Label(playback_frame, text="Frame Rate (FPS):").pack(anchor=tk.W)
        ttk.Spinbox(playback_frame, from_=1, to=60, textvariable=self.frame_rate, width=5).pack(anchor=tk.W, pady=(0, 5))
        
        # Playback buttons
        button_frame = ttk.Frame(playback_frame)
        button_frame.pack(fill=tk.X)
        
        ttk.Button(button_frame, text="⏮", width=3, command=self.go_to_first_frame).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏪", width=3, command=self.previous_frame).pack(side=tk.LEFT)
        self.play_button = ttk.Button(button_frame, text="▶", width=3, command=self.toggle_playback)
        self.play_button.pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏩", width=3, command=self.next_frame).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏭", width=3, command=self.go_to_last_frame).pack(side=tk.LEFT)
        
        # Export section
        export_frame = ttk.LabelFrame(control_frame, text="Export", padding="5")
        export_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(export_frame, text="Export as GIF", command=self.export_as_gif).pack(fill=tk.X)
        ttk.Button(export_frame, text="Export as MP4", command=self.export_as_mp4).pack(fill=tk.X, pady=(5, 0))
        ttk.Button(export_frame, text="Export Frames", command=self.export_frames).pack(fill=tk.X, pady=(5, 0))
        
        # Status bar
        status_bar = ttk.Label(root, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Bind keyboard shortcuts
        self.root.bind("<Left>", lambda e: self.previous_frame())
        self.root.bind("<Right>", lambda e: self.next_frame())
        self.root.bind("<space>", lambda e: self.toggle_playback())
        
        # Initialize the display
        self.display_placeholder()
    
    def load_contact_sheet(self):
        """Load a contact sheet image."""
        try:
            # Use tkinter file dialog with explicit initialdir to avoid None issues
            file_path = filedialog.askopenfilename(
                title="Select Contact Sheet",
                initialdir=".",  # Start in current directory
                filetypes=[
                    ("Image files", "*.png *.jpg *.jpeg *.tif *.tiff *.bmp"),
                    ("PDF files", "*.pdf"),
                    ("All files", "*.*")
                ]
            )
            
            # Check if file_path is not empty (user didn't cancel)
            if file_path and os.path.exists(file_path):
                try:
                    # Load the image
                    if file_path.lower().endswith('.pdf'):
                        # Convert PDF to image
                        try:
                            from pdf2image import convert_from_path
                            images = convert_from_path(file_path, dpi=300, first_page=1, last_page=1)
                            if images and len(images) > 0:
                                self.contact_sheet = images[0]
                            else:
                                raise ValueError("Failed to convert PDF to image")
                        except ImportError:
                            # Fallback to using pdftoppm via subprocess
                            with tempfile.TemporaryDirectory() as temp_dir:
                                temp_prefix = os.path.join(temp_dir, "pdf_page")
                                result = subprocess.run(["pdftoppm", "-png", "-f", "1", "-l", "1", file_path, temp_prefix], 
                                                    capture_output=True, check=False)
                                
                                if result.returncode != 0:
                                    raise ValueError(f"PDF conversion failed: {result.stderr.decode('utf-8', errors='ignore')}")
                                    
                                # Find the generated PNG file
                                png_files = [f for f in os.listdir(temp_dir) if f.endswith(".png")]
                                if not png_files:
                                    raise ValueError("No PNG files generated from PDF")
                                    
                                img_path = os.path.join(temp_dir, png_files[0])
                                self.contact_sheet = Image.open(img_path)
                    else:
                        # Load regular image file
                        self.contact_sheet = Image.open(file_path)
                    
                    # Update status
                    self.status_var.set(f"Loaded contact sheet: {os.path.basename(file_path)}")
                    
                    # Display the contact sheet
                    self.display_contact_sheet()
                    
                    # Clear any existing frames
                    self.extracted_frames = []
                    self.current_frame_index = 0
                    
                except Exception as e:
                    messagebox.showerror("Error", f"Error loading contact sheet: {str(e)}")
                    
            elif file_path:  # Path selected but doesn't exist
                messagebox.showerror("Error", f"File not found: {file_path}")
                
        except Exception as e:
            # Catch any exceptions in the file dialog itself
            messagebox.showerror("Error", f"File dialog error: {str(e)}")
            print(f"File dialog error: {str(e)}")
    ##
    def extract_frames_from_registration_marks(self):
        """Extract frames from a contact sheet using registration mark detection."""
        if self.contact_sheet is None:
            messagebox.showwarning("Warning", "No contact sheet loaded")
            return
                
        try:
            # Convert PIL Image to OpenCV format
            cv_image = cv2.cvtColor(np.array(self.contact_sheet), cv2.COLOR_RGB2BGR)
            
            # Create frame extractor
            extractor = PreciseFrameExtractor()
            
            # Set debug mode if needed
            extractor.set_debug_mode(self.debug_mode.get())
            
            # Extract frames - this will now prioritize registration mark detection
            frames = extractor.extract_frames(cv_image, self.frames_per_row.get())
            
            if not frames:
                messagebox.showwarning("Warning", "No frames could be extracted")
                return
            
            # Convert OpenCV frames back to PIL format
            pil_frames = []
            for frame in frames:
                # Verify frame is valid before conversion
                if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                    print("Skipping invalid frame during conversion")
                    continue
                    
                try:
                    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    pil_frames.append(pil_frame)
                except Exception as e:
                    print(f"Error converting frame: {str(e)}")
                    continue
            
            # Update the extracted frames
            self.extracted_frames = pil_frames
            self.current_frame_index = 0
            
            # Display the first frame if available
            if self.extracted_frames:
                self.display_current_frame()
                # Update status
                self.status_var.set(f"Successfully extracted {len(pil_frames)} frames")
            else:
                messagebox.showwarning("Warning", "No valid frames could be extracted")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error extracting frames: {str(e)}")
            import traceback
            traceback.print_exc()

    
    def extract_frames_from_registration_marks(self):
        """Extract frames from a contact sheet using registration mark detection."""
        if self.contact_sheet is None:
            messagebox.showwarning("Warning", "No contact sheet loaded")
            return
                
        try:
            # Convert PIL Image to OpenCV format
            cv_image = cv2.cvtColor(np.array(self.contact_sheet), cv2.COLOR_RGB2BGR)
            
            # Create frame extractor
            extractor = PreciseFrameExtractor()
            
            # Set debug mode if needed
            extractor.set_debug_mode(self.debug_mode.get())
            
            # Extract frames
            frames = extractor.extract_frames(cv_image, self.frames_per_row.get())
            
            if not frames:
                messagebox.showwarning("Warning", "No frames could be extracted")
                return
            
            # Convert OpenCV frames back to PIL format
            pil_frames = []
            for frame in frames:
                # Verify frame is valid before conversion
                if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                    print("Skipping invalid frame during conversion")
                    continue
                    
                try:
                    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    pil_frames.append(pil_frame)
                except Exception as e:
                    print(f"Error converting frame: {str(e)}")
                    continue
            
            # Update the extracted frames
            self.extracted_frames = pil_frames
            self.current_frame_index = 0
            
            # Display the first frame if available
            if self.extracted_frames:
                self.display_current_frame()
                # Update status
                self.status_var.set(f"Successfully extracted {len(pil_frames)} frames")
            else:
                messagebox.showwarning("Warning", "No valid frames could be extracted")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error extracting frames: {str(e)}")
            import traceback
            traceback.print_exc()

    def display_current_frame(self):
        """Display the current frame"""
        if not self.extracted_frames or self.current_frame_index >= len(self.extracted_frames):
            self.display_placeholder()
            return
        
        # Get the current frame
        frame = self.extracted_frames[self.current_frame_index]
        
        # Resize frame to fit canvas while maintaining aspect ratio
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        if canvas_width <= 1 or canvas_height <= 1:
            # Canvas not yet properly sized, use default size
            canvas_width = 400
            canvas_height = 400
        
        # Calculate scale factor to fit frame within canvas
        frame_width, frame_height = frame.size
        scale_width = canvas_width / frame_width
        scale_height = canvas_height / frame_height
        scale = min(scale_width, scale_height)
        
        # Resize frame
        new_width = int(frame_width * scale)
        new_height = int(frame_height * scale)
        resized_frame = frame.resize((new_width, new_height), Image.LANCZOS)
        
        # Convert to PhotoImage
        self.photo = ImageTk.PhotoImage(resized_frame)
        
        # Clear canvas and display image
        self.canvas.delete("all")
        
        # Calculate position to center image on canvas
        x = (canvas_width - new_width) // 2
        y = (canvas_height - new_height) // 2
        
        # Create image on canvas
        self.canvas.create_image(x, y, anchor=tk.NW, image=self.photo)
        
        # Add frame number text
        frame_text = f"Frame {self.current_frame_index + 1}/{len(self.extracted_frames)}"
        self.canvas.create_text(canvas_width // 2, 20, text=frame_text, fill="black", font=("Arial", 12))
    
    def display_contact_sheet(self):
        """Display the loaded contact sheet"""
        if self.contact_sheet is None:
            self.display_placeholder()
            return
        
        # Resize contact sheet to fit canvas while maintaining aspect ratio
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        if canvas_width <= 1 or canvas_height <= 1:
            # Canvas not yet properly sized, use default size
            canvas_width = 400
            canvas_height = 400
        
        # Calculate scale factor to fit contact sheet within canvas
        sheet_width, sheet_height = self.contact_sheet.size
        scale_width = canvas_width / sheet_width
        scale_height = canvas_height / sheet_height
        scale = min(scale_width, scale_height)
        
        # Resize contact sheet
        new_width = int(sheet_width * scale)
        new_height = int(sheet_height * scale)
        resized_sheet = self.contact_sheet.resize((new_width, new_height), Image.LANCZOS)
        
        # Convert to PhotoImage
        self.photo = ImageTk.PhotoImage(resized_sheet)
        
        # Clear canvas and display image
        self.canvas.delete("all")
        
        # Calculate position to center image on canvas
        x = (canvas_width - new_width) // 2
        y = (canvas_height - new_height) // 2
        
        # Create image on canvas
        self.canvas.create_image(x, y, anchor=tk.NW, image=self.photo)
        
        # Add contact sheet text
        self.canvas.create_text(canvas_width // 2, 20, text="Contact Sheet", fill="black", font=("Arial", 12))
    
    def display_placeholder(self):
        """Display a placeholder when no image is loaded"""
        # Clear canvas
        self.canvas.delete("all")
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        if canvas_width <= 1 or canvas_height <= 1:
            # Canvas not yet properly sized, use default size
            canvas_width = 400
            canvas_height = 400
        
        # Create placeholder text
        self.canvas.create_text(
            canvas_width // 2,
            canvas_height // 2,
            text="No image loaded",
            fill="gray",
            font=("Arial", 14)
        )
    
    def go_to_first_frame(self):
        """Go to the first frame"""
        if self.extracted_frames:
            self.current_frame_index = 0
            self.display_current_frame()
    
    def previous_frame(self):
        """Go to the previous frame"""
        if self.extracted_frames and self.current_frame_index > 0:
            self.current_frame_index -= 1
            self.display_current_frame()
    
    def next_frame(self):
        """Go to the next frame"""
        if self.extracted_frames and self.current_frame_index < len(self.extracted_frames) - 1:
            self.current_frame_index += 1
            self.display_current_frame()
    
    def go_to_last_frame(self):
        """Go to the last frame"""
        if self.extracted_frames:
            self.current_frame_index = len(self.extracted_frames) - 1
            self.display_current_frame()
    
    def toggle_playback(self):
        """Toggle animation playback"""
        if not self.extracted_frames:
            return
        
        if self.is_playing:
            # Stop playback
            self.is_playing = False
            self.play_button.configure(text="▶")
            
            # Cancel scheduled frame update
            if self.play_after_id:
                self.root.after_cancel(self.play_after_id)
                self.play_after_id = None
        else:
            # Start playback
            self.is_playing = True
            self.play_button.configure(text="⏸")
            
            # Schedule first frame update
            self.play_animation()
    
    def play_animation(self):
        """Play the animation"""
        if not self.is_playing or not self.extracted_frames:
            return
        
        # Display current frame
        self.display_current_frame()
        
        # Move to next frame or loop back to start
        if self.current_frame_index < len(self.extracted_frames) - 1:
            self.current_frame_index += 1
        else:
            self.current_frame_index = 0
        
        # Calculate delay based on frame rate
        delay_ms = int(1000 / self.frame_rate.get())
        
        # Schedule next frame update
        self.play_after_id = self.root.after(delay_ms, self.play_animation)
    
    def export_as_gif(self):
        """Export the extracted frames as a GIF animation"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        # Ask for save location
        file_path = filedialog.asksaveasfilename(
            title="Save GIF As",
            defaultextension=".gif",
            filetypes=[("GIF files", "*.gif")]
        )
        
        if not file_path:
            return
        
        try:
            # Calculate frame duration based on frame rate
            duration_ms = int(1000 / self.frame_rate.get())
            
            # Save frames as GIF
            self.extracted_frames[0].save(
                file_path,
                save_all=True,
                append_images=self.extracted_frames[1:],
                optimize=False,
                duration=duration_ms,
                loop=0
            )
            
            self.status_var.set(f"Saved GIF to {os.path.basename(file_path)}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error saving GIF: {str(e)}")
    
    def export_as_mp4(self):
        """Export the extracted frames as an MP4 video"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        # Ask for save location
        file_path = filedialog.asksaveasfilename(
            title="Save MP4 As",
            defaultextension=".mp4",
            filetypes=[("MP4 files", "*.mp4")]
        )
        
        if not file_path:
            return
        
        try:
            # Create a temporary directory for frame images
            with tempfile.TemporaryDirectory() as temp_dir:
                # Save frames as PNG files
                for i, frame in enumerate(self.extracted_frames):
                    frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
                    frame.save(frame_path)
                
                # Get frame rate
                fps = self.frame_rate.get()
                
                # Use FFmpeg to create MP4
                cmd = [
                    "ffmpeg",
                    "-y",  # Overwrite output file if it exists
                    "-framerate", str(fps),
                    "-i", os.path.join(temp_dir, "frame_%04d.png"),
                    "-c:v", "libx264",
                    "-pix_fmt", "yuv420p",
                    "-crf", "23",  # Quality (lower is better)
                    file_path
                ]
                
                # Run FFmpeg
                subprocess.run(cmd, check=True)
                
                self.status_var.set(f"Saved MP4 to {os.path.basename(file_path)}")
                
        except Exception as e:
            messagebox.showerror("Error", f"Error saving MP4: {str(e)}")
    
    def export_frames(self):
        """Export individual frames as image files"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        # Ask for save directory
        save_dir = filedialog.askdirectory(title="Select Directory to Save Frames")
        
        if not save_dir:
            return
        
        try:
            # Save each frame
            for i, frame in enumerate(self.extracted_frames):
                frame_path = os.path.join(save_dir, f"frame_{i+1:04d}.png")
                frame.save(frame_path)
            
            self.status_var.set(f"Saved {len(self.extracted_frames)} frames to {os.path.basename(save_dir)}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error saving frames: {str(e)}")


# Function to run the application
def run_app():
    root = tk.Tk()
    app = AnimationApp(root)
    root.geometry("800x600")
    root.mainloop()

# Run the application when executed directly
if __name__ == "__main__":
    run_app()


2025-03-25 21:09:01.971 Python[22890:1273330] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-25 21:09:01.971 Python[22890:1273330] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Attempting registration mark detection...
Registration mark detection failed: Not enough registration marks detected (found 0)
Trying grid-based extraction...
Grid-based extraction failed: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

Attempting content-based extraction...
Successfully extracted 36 frames using content-based approach
Attempting registration mark detection...
Registration mark detection failed: Not enough registration marks detected (found 0)
Trying grid-based extraction...
Successfully extracted 35 frames using grid-based approach
Attempting registration mark detection...
Registration mark detection failed: Not enough registration marks detected (found 1)
Trying grid-based extraction...
Grid-based extraction failed: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc